<a href="https://colab.research.google.com/github/lucianosilva-github/visaocomputacional/blob/master/Aula%2021/Visa%CC%83o_Computacional_Aula_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **VISÃO COMPUTACIONAL - AULA 21**

**Objetivos da aula:**

*   apresentar o conceito de busca por padrões 3D em nuvens de pontos
*   apresentar um algoritmo de ICP para fazer busca por registros
*   implementar e praticar com algoritm de ICP


## **BUSCA POR PADRÕES 3D EM NUVENS DE PONTOS**

A busca semântica em nuvens de pontos é um recurso moderno em Visão Computacional 3D e permite que padrões 3D (também definidos como nuvens de pontos) sejam procurados em outras nuvens de pontos. Abaixo temos um exemplo de padrão (amarelo) sendo procurado em uma nuvem de pontos (azul).

<img src="https://lh3.googleusercontent.com/proxy/QtWMtYLRDuk9Zzg_J6KyIxcobTZvpsIcIfvUsFbW3WT0q_ikcTGdZbhuanaWbSJJcxKxC5-8swPN-tLpQerAFu0Awdw04c2EDpuDL5xkBg73F_gw-czDue-4SfLxaakXnQWiLj95waYqZA"> 
</img>

Este procedimento é chamado registro ICP (Iterative Closest Point). Existem diversos algoritmos de registro ICP. Um estudo comparativo bastante completo destes algoritmos pode ser encontrado no artigo abaixo:

S. Rusinkiewicz and M. Levoy. **Efficient variants of the ICP algorithm**. In 3-D Digital Imaging and Modeling, 2001.

O módulo Open3D possui um suporte bastante efetivo para ICP. Trabalharemos com dois algoritmos de ICP implementandos na Open3D: ICP Ponto-a-Ponto e o ICP Ponto-a-Plano. Para usar estes dois algoritmos, vamos utilizar dois arquivos de nuvens de pontos: um para o espaço de busca e outro para o padrão a ser encontrado.

In [ ]:
!pip install open3d

Abertura e exibição do espaço de busca.

In [ ]:
import open3d as o3d
import numpy as np

pcdb = o3d.io.read_point_cloud("espaco_busca.pcd")
print(pcdb)
points=np.asarray(pcdb.points)
print(points)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=1, color=np.asarray(pcd.colors))
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

## **EXERCÍCIO**

Leia e visualize a nuvem de pontos bunny.ply:

In [ ]:
#digite sua solução aqui

## **ESTIMAÇÃO DE MALHA (MESH) DE UMA NUVEM DE PONTOS**

Existem diversos algoritmos para reconstruir malhas a partir de nuvens de pontos. Um dos mais simples e utilizados é o algoritmo BPA (Ball-Pivoting Algorithm): 

*F. Bernardini and J. Mittleman and HRushmeier and C. Silva and G. Taubin: The ball-pivoting algorithm for surface reconstruction, IEEE transactions on visualization and computer graphics, 5(4), 349-359, 1999.*

Link para o artigo: https://vgc.poly.edu/~csilva/papers/tvcg99.pdf 


## **EXERCÍCIO**

Ler e explicar como funciona o algoritmo BPA.

In [ ]:
#explique como funciona o algoritmo BPA aqui

Conforme visto no artigo acima, o algoritmo depende de 4 (quatro) raios, especificados pela lista de raios radii:


In [ ]:
pcd = o3d.io.read_point_cloud("bunny.ply")
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
radii = [0.005, 0.01, 0.02, 0.04]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))

A partir da malha estimada, estimamos também as normais dos vértices e dos triângulos que compõem a malha:

In [ ]:
if not mesh.has_vertex_normals(): 
  mesh.compute_vertex_normals()
if not mesh.has_triangle_normals(): 
  mesh.compute_triangle_normals()

Se a malha não possui cores em suas faces, adicionamos cores artificiais:

In [ ]:
triangles = np.asarray(mesh.triangles)
vertices = np.asarray(mesh.vertices)
colors = None
if mesh.has_triangle_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
    colors = tuple(map(tuple, colors))
else:
    colors = (1.0, 0.0, 0.0)

Finalmente podemos desenhar a malha:

In [ ]:
fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            facecolor=colors,
            opacity=0.50)
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

## **EXERCÍCIO**

Variar os raios dos quatro círculos e observar a qualidade da reconstrução da malha.

In [ ]:
#faça seus experimentos aqui

## **EXERCÍCIO**

Reconstrua a malha da nuvem de pontos points.ply, com diferentes raios do algoritmo BPA.

In [ ]:
#implementa sua solução aqui